## Informações Gerais
 | Informações | Detalhes |
 |------------|-------------|
 |Nome Tabela | silver.pedidos |
 |Origem | bronze.pedidos / bronze.estabelecimentos |

## Histórico de Atualizações
 | Data | Desenvolvido por | Motivo |
 |:----:|--------------|--------|
 |06/09/2024 | Daniel Coutinho  | Criação do notebook |
 |13/09/2024 | Daniel Coutinho  | Inclusão da coluna "valor" |

In [0]:
%run /Projeto_Ampev/00.config/Configurações

In [0]:
# Importações
from pyspark.sql.functions import current_date, current_timestamp, expr

In [0]:
# Funções
def adicionaComentariosTabela(a,b,c,d):
    spark.sql(f"COMMENT ON TABLE {a}.{b} IS '{c}'")
    for key,value in d.items():
        sqlaux = f"ALTER TABLE {a}.{b} CHANGE COLUMN {key} COMMENT '{value}'"
        spark.sql(sqlaux)

In [0]:
database = "silver"
tabela = "pedidos"

In [0]:
# Criação do dataframe df_pedidos utilizando CTE com JOIN entre as tabelas pedidos e estabelecimentos para uma nova tabela

df_pedidos = spark.sql(
f"""
WITH pedidos
AS (
	SELECT PedidoID AS id_pedido
		,EstabelecimentoID AS id_estabelecimento
		,Produto AS produto
		,quantidade_vendida AS quantidade
		,Preco_Unitario AS preco
		,data_venda AS data_pedido
	FROM bronze.pedidos
	)
	,estabelecimentos
AS (
	SELECT LOCAL AS nome_estabelecimento
		,Email AS email
		,EstabelecimentoID AS id_estabelecimento
		,Telefone AS telefone
	FROM bronze.estabelecimentos
	)
SELECT CAST(p.id_pedido AS INT)
	,CAST(p.id_estabelecimento AS INT)
	,p.produto
	,CAST(p.quantidade AS INT)
	,CAST(p.preco AS DECIMAL(20,2))
	,e.nome_estabelecimento
	,e.email
	,e.telefone
	,CAST(p.data_pedido AS DATE)
FROM pedidos p
LEFT JOIN estabelecimentos e ON p.id_estabelecimento = e.id_estabelecimento
"""
)

In [0]:
# incluir colunas de controle
df_pedidos = df_pedidos.withColumn("data_carga", current_date())
df_pedidos = df_pedidos.withColumn("data_hora_carga", expr("current_timestamp() - INTERVAL 3 HOURS"))

In [0]:
# Comentário Tabela
comentario_tabela = 'Esta tabela é uma entidade corporativa que contém a relação dos pedidos'

lista_comentario_colunas = {
    'id_pedido' : 'Id do pedido.',
    'id_estabelecimento' :  'Id do estabelecimento.',
    'produto' : 'Nome do produto.',
    'quantidade' : 'Quantidade do pedido.',
    'preco' : 'Preço do produto unitario.',
    'nome_estabelecimento' : 'Nome do estabelecimento.',
    'email' :  'E-mail do estabelecimento.',
    'telefone' : 'Telefone do estabelecimento.',
    'data_pedido' : 'Data do pedido.',
    'data_carga' : 'Data que o registro foi carregado',
    'data_hora_carga' : 'Data e hora que o registro foi carregado'
}

In [0]:
# Grava os dados no formato Delta (utiliza liquid cluster para otimização)
df_pedidos.write \
          .format("delta") \
          .mode("overwrite") \
          .clusterBy("id_pedido") \
          .option("overwriteSchema", "true") \
          .saveAsTable(f"{database}.{tabela}")
adicionaComentariosTabela(database, tabela, comentario_tabela, lista_comentario_colunas)
print("Dados gravados com sucesso!")

Dados gravados com sucesso!


In [0]:
spark.sql(f"OPTIMIZE {database}.{tabela}")
print("Otimização finalizada!")

#Otimiza o layout dos dados do Delta Lake

Otimização finalizada!


In [0]:
%sql
SELECT *
FROM silver.pedidos

id_pedido,id_estabelecimento,produto,quantidade,preco,nome_estabelecimento,email,telefone,data_pedido,data_carga,data_hora_carga
148,38,Cerveja XYZ,238,1.30,Restaurante Francês,contato@restaurantefrances.com,(21) 91283-5782,2023-07-11,2024-09-19,2024-09-19T19:43:06.172Z
243,23,Água Mineral,337,18.82,Academia Fit,contato@academiafit.com,(11) 92674-3789,2024-04-22,2024-09-19,2024-09-19T19:43:06.172Z
31,2,Suco Natural,439,10.89,Restaurante da Ana,ana@restaurantedana.com,(11) 98891-8293,2023-11-10,2024-09-19,2024-09-19T19:43:06.172Z
85,19,Água Mineral,177,4.37,Barbearia do Toninho,contato@barbeariadotoninho.com,(91) 99438-6690,2023-11-22,2024-09-19,2024-09-19T19:43:06.172Z
137,36,Vinho ABC,180,5.32,Padaria do Bairro,contato@padariadobairro.com,(31) 95389-1464,2023-08-01,2024-09-19,2024-09-19T19:43:06.172Z
65,36,Água Mineral,220,11.01,Padaria do Bairro,contato@padariadobairro.com,(31) 95389-1464,2024-01-08,2024-09-19,2024-09-19T19:43:06.172Z
53,24,Refrigerante 123,480,12.53,Salão Beleza Pura,contato@salaobelezapura.com,(51) 91844-9553,2024-02-09,2024-09-19,2024-09-19T19:43:06.172Z
133,44,Refrigerante 123,174,12.47,Lanchonete do Bairro,contato@lanchonetedobairro.com,(61) 93857-1485,2023-10-19,2024-09-19,2024-09-19T19:43:06.172Z
78,20,Água Mineral,212,12.08,Farmácia Popular,contato@farmaciapopular.com,(51) 94871-3208,2023-12-24,2024-09-19,2024-09-19T19:43:06.172Z
108,4,Vinho ABC,307,14.69,Padaria do Paulo,paulo@padariapaulo.com,(61) 97029-4259,2023-10-12,2024-09-19,2024-09-19T19:43:06.172Z


In [0]:
%sql
DESC DETAIL silver.pedidos

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,45195348-0763-420c-bfa9-b5f37c8f5e85,spark_catalog.silver.pedidos,Esta tabela é uma entidade corporativa que contém a relação dos pedidos,dbfs:/user/hive/warehouse/dbfs/FileStore/Projeto_Ampev/silver/pedidos,2024-09-14T02:44:49.101Z,2024-09-19T22:44:35Z,List(),List(id_pedido),1,11022,"Map(delta.enableDeletionVectors -> true, delta.enableRowTracking -> true, delta.checkpointPolicy -> v2, delta.rowTracking.materializedRowCommitVersionColumnName -> _row-commit-version-col-5162ba75-df5a-4def-90d9-b9ddd24b5662, delta.rowTracking.materializedRowIdColumnName -> _row-id-col-fcf95a97-3300-4128-8eae-620510c307d7)",3,7,"List(clustering, deletionVectors, domainMetadata, rowTracking, v2Checkpoint)","Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)"
